In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install wurlitzer

In [3]:
%load_ext wurlitzer

In [4]:
import pandas as pd
import numpy as np
import logging
logging.basicConfig(level = logging.INFO)

In [5]:
import pyfdb
import findlibs
import yaml
from pathlib import Path
import os
import pandas as pd
import pyodc
import shutil

fdb5_path = Path(findlibs.find("fdb5"))
metkit_path = Path(findlibs.find("metkit"))
print(f"Using fdb5 shared library from {fdb5_path}")
print(f"Using metkit shared library from {metkit_path}")

marsrequest = (metkit_path.parents[1] / "share/metkit/odb/marsrequest.yaml").resolve()
languages = (metkit_path.parents[1] / "share/metkit/language.yaml").resolve() 

print(f"\nContents of {marsrequest}:\n", marsrequest.open().read())

fields = " ".join(yaml.safe_load(languages.open().read())["_field"].keys())
print(f"Fields defined in {languages}:\n {fields}")

Using fdb5 shared library from /Users/math/micromamba/envs/ionbeam/lib/libfdb5.dylib
Using metkit shared library from /Users/math/micromamba/envs/ionbeam/lib/libmetkit.dylib

Contents of /Users/math/micromamba/envs/ionbeam/share/metkit/odb/marsrequest.yaml:
 ---
PLATFORM: platform
OBSERVATION_VARIABLE: observation_variable
Fields defined in /Users/math/micromamba/envs/ionbeam/share/metkit/language.yaml:
 observation_variable platform class type stream expver dataset model repres obsgroup reportype levtype levelist param date hdate offsetdate fcmonth fcperiod time offsettime step anoffset reference number quantile domain frequency direction diagnostic iteration channel ident instrument method origin system


In [6]:
from pathlib import Path
from ionbeam.core.config_parser import parse_config
from ionbeam.core.bases import *
from IPython.display import display

config_file = Path("~/git/IonBeam-Deployment/config/ionbeam").expanduser()
config, actions = parse_config(config_file,
                    config_path = "./",
                    data_path = "../data/",
                    offline = True,
                    environment  = "local",
                    )

ConfigError: While parsing key='actions', _type=typing.List[ionbeam.core.bases.Action], value=[{'class': 'SmartCitizenKitSource', 'copy_metadata_to_columns': [{'name': 'sensor.name', 'column': 'sensor_name', '__line__': 4}], '__line__': 2}], got exception 
		While parsing key='element 0', _type=<class 'ionbeam.core.bases.Action'>, value={'class': 'SmartCitizenKitSource', 'copy_metadata_to_columns': [{'name': 'sensor.name', 'column': 'sensor_name', '__line__': 4}], '__line__': 2}, got exception 
		While parsing key='copy_metadata_to_columns', _type=list[ionbeam.core.bases.InputColumn], value=[{'name': 'sensor.name', 'column': 'sensor_name', '__line__': 4}], got exception 
		While parsing key='element 0', _type=<class 'ionbeam.core.bases.InputColumn'>, value={'name': 'sensor.name', 'column': 'sensor_name', '__line__': 4}, got exception 
		Invalid config yaml entry
        /Users/math/git/IonBeam-Deployment/config/ionbeam/smart_citizen_kit/actions.yaml:4
        Determined the type to be 

In [7]:
from datetime import datetime as dt
from datetime import timezone, timedelta


config, actions = parse_config(config_file,
                    config_path = "./",
                    data_path = "../data/",
                    offline = True,
                    environment  = "local",
                    )

def make_fake_message(i, start):
    times = pd.date_range(start=start, end=start + timedelta(hours = 1, minutes = 59), 
                          freq = "30min", tz = timezone.utc)
    return TabularMessage(
        metadata=MetaData(state='foo',
            source='meteotracker',
            observation_variable = "bar",
                         
        ),
        data = pd.DataFrame(dict(
            time = times,
            data = [i for _ in times],
            
        ))
    )

agg = actions[3]
for i, start_time in enumerate(pd.date_range(start=dt(2024, 1, 1, 0), end=dt(2024, 1, 3, 0),
                                    freq = "1h", tz = timezone.utc)):

    print(f"Putting in message with {start_time = }")
    for m in agg.process(make_fake_message(i, start_time, )):
        print("Time agg emitted", m, m.data)

{'database': 'ionbeam', 'username': 'postgres', 'password': 'g7HEuiMHJTKczM4k', 'host': 'localhost', 'port': 5432}
Putting in message with start_time = Timestamp('2024-01-01 00:00:00+0000', tz='UTC')
Putting in message with start_time = Timestamp('2024-01-01 01:00:00+0000', tz='UTC')
Putting in message with start_time = Timestamp('2024-01-01 02:00:00+0000', tz='UTC')
Putting in message with start_time = Timestamp('2024-01-01 03:00:00+0000', tz='UTC')
Putting in message with start_time = Timestamp('2024-01-01 04:00:00+0000', tz='UTC')
Putting in message with start_time = Timestamp('2024-01-01 05:00:00+0000', tz='UTC')
Putting in message with start_time = Timestamp('2024-01-01 06:00:00+0000', tz='UTC')
Putting in message with start_time = Timestamp('2024-01-01 07:00:00+0000', tz='UTC')
Putting in message with start_time = Timestamp('2024-01-01 08:00:00+0000', tz='UTC')
Putting in message with start_time = Timestamp('2024-01-01 09:00:00+0000', tz='UTC')
Putting in message with start_time 